# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc57c5efd90>
├── 'a' --> tensor([[ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059]])
└── 'x' --> <FastTreeValue 0x7fc57c5f6730>
    └── 'c' --> tensor([[-1.5481, -1.1210, -0.4121, -0.4152],
                        [-1.8367, -1.0638,  0.1189,  0.8731],
                        [ 2.8579,  0.6689,  1.0275,  0.6867]])

In [4]:
t.a

tensor([[ 0.3558,  0.5405, -0.0064],
        [-0.6658, -1.2103, -1.0059]])

In [5]:
%timeit t.a

63.2 ns ± 0.0453 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc57c5efd90>
├── 'a' --> tensor([[ 0.4973,  0.8735,  1.0416],
│                   [ 0.0574,  0.5433, -1.0516]])
└── 'x' --> <FastTreeValue 0x7fc57c5f6730>
    └── 'c' --> tensor([[-1.5481, -1.1210, -0.4121, -0.4152],
                        [-1.8367, -1.0638,  0.1189,  0.8731],
                        [ 2.8579,  0.6689,  1.0275,  0.6867]])

In [7]:
%timeit t.a = new_value

61.3 ns ± 0.0341 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059]]),
    x: Batch(
           c: tensor([[-1.5481, -1.1210, -0.4121, -0.4152],
                      [-1.8367, -1.0638,  0.1189,  0.8731],
                      [ 2.8579,  0.6689,  1.0275,  0.6867]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3558,  0.5405, -0.0064],
        [-0.6658, -1.2103, -1.0059]])

In [11]:
%timeit b.a

51.3 ns ± 0.0324 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2437, -1.2994,  0.7367],
               [ 0.8839,  0.2655, -2.3785]]),
    x: Batch(
           c: tensor([[-1.5481, -1.1210, -0.4121, -0.4152],
                      [-1.8367, -1.0638,  0.1189,  0.8731],
                      [ 2.8579,  0.6689,  1.0275,  0.6867]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

850 ns ± 0.494 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 18.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 598 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc4cd21f4f0>
├── 'a' --> tensor([[[ 0.3558,  0.5405, -0.0064],
│                    [-0.6658, -1.2103, -1.0059]],
│           
│                   [[ 0.3558,  0.5405, -0.0064],
│                    [-0.6658, -1.2103, -1.0059]],
│           
│                   [[ 0.3558,  0.5405, -0.0064],
│                    [-0.6658, -1.2103, -1.0059]],
│           
│                   [[ 0.3558,  0.5405, -0.0064],
│                    [-0.6658, -1.2103, -1.0059]],
│           
│                   [[ 0.3558,  0.5405, -0.0064],
│                    [-0.6658, -1.2103, -1.0059]],
│           
│                   [[ 0.3558,  0.5405, -0.0064],
│                    [-0.6658, -1.2103, -1.0059]],
│           
│                   [[ 0.3558,  0.5405, -0.0064],
│                    [-0.6658, -1.2103, -1.0059]],
│           
│                   [[ 0.3558,  0.5405, -0.0064],
│                    [-0.6658, -1.2103, -1.0059]]])
└── 'x' --> <FastTreeValue 0x7fc4cd21f8e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 60.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc4cd2410a0>
├── 'a' --> tensor([[ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059],
│                   [ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059],
│                   [ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059],
│                   [ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059],
│                   [ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059],
│                   [ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059],
│                   [ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059],
│                   [ 0.3558,  0.5405, -0.0064],
│                   [-0.6658, -1.2103, -1.0059]])
└── 'x' --> <FastTreeValue 0x7fc4cd296520>
    └── 'c' --> tensor([[-1.5481, -1.1210, -0.4121, -0.4152],
                        [-1.8367, -1.0638,  0.1189,  0.8731],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 60.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.4 µs ± 43.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.3558,  0.5405, -0.0064],
                [-0.6658, -1.2103, -1.0059]],
       
               [[ 0.3558,  0.5405, -0.0064],
                [-0.6658, -1.2103, -1.0059]],
       
               [[ 0.3558,  0.5405, -0.0064],
                [-0.6658, -1.2103, -1.0059]],
       
               [[ 0.3558,  0.5405, -0.0064],
                [-0.6658, -1.2103, -1.0059]],
       
               [[ 0.3558,  0.5405, -0.0064],
                [-0.6658, -1.2103, -1.0059]],
       
               [[ 0.3558,  0.5405, -0.0064],
                [-0.6658, -1.2103, -1.0059]],
       
               [[ 0.3558,  0.5405, -0.0064],
                [-0.6658, -1.2103, -1.0059]],
       
               [[ 0.3558,  0.5405, -0.0064],
                [-0.6658, -1.2103, -1.0059]]]),
    x: Batch(
           c: tensor([[[-1.5481, -1.1210, -0.4121, -0.4152],
                       [-1.8367, -1.0638,  0.1189,  0.8731],
                       [ 2.8579,  0.6689,  1.0275,  0.6867]],
         

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 168 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059],
               [ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059],
               [ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059],
               [ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059],
               [ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059],
               [ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059],
               [ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059],
               [ 0.3558,  0.5405, -0.0064],
               [-0.6658, -1.2103, -1.0059]]),
    x: Batch(
           c: tensor([[-1.5481, -1.1210, -0.4121, -0.4152],
                      [-1.8367, -1.0638,  0.1189,  0.8731],
                      [ 2.8579,  0.6689,  1.0275,  0.6867],
                      [-1.5481, -1.1210, -0.4121, -0.4152],
                      [-1.8367, -

In [28]:
%timeit Batch.cat(batches)

172 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
